In [5]:
import folium
import requests
import json
import math
import time
from PIL import Image
from io import BytesIO

In [6]:
def get_json_data(url):
    # JSONデータの取得
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to get data from {url}: {response.status_code}")
        return None

In [7]:
# 最新情報の取得
target_url = "https://www.jartic.or.jp/d/traffic_info/r1/target.json"
target_data = get_json_data(target_url)

In [8]:
target_data

{'target': '202410291322'}

In [51]:
 # 各交通情報の取得
target = target_data.get("target", "")
infos = ["R13", "C01", "A03"]
for route in infos:
    traffic_url = "https://www.jartic.or.jp/d/traffic_info/r2/202410291411/d/900/R20.json"
    traffic_data = get_json_data(traffic_url)

In [52]:
traffic_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 1,
   'properties': {'ii': '972007000030',
    'r': '国道１８号',
    'ad': '長野県北佐久郡軽井沢町大字追分',
    'ot': '2024年10月29日 13:55',
    'ci': True,
    'pi': '972007000030_20241029_1355.jpg',
    'sn': False,
    'vs': '軽井沢追分\u3000上田方向'},
   'geometry': {'type': 'Point',
    'crs': {'type': 'name', 'properties': {'name': 'EPSG:4612'}},
    'coordinates': [138.55578, 36.34239]}},
  {'type': 'Feature',
   'id': 2,
   'properties': {'ii': '972007000010',
    'r': '国道１８号',
    'ad': '長野県北佐久郡軽井沢町大字長倉',
    'ot': '2024年10月29日 14:00',
    'ci': True,
    'pi': '972007000010_20241029_1400.jpg',
    'sn': False,
    'vs': '軽井沢町長倉\u3000上田方向'},
   'geometry': {'type': 'Point',
    'crs': {'type': 'name', 'properties': {'name': 'EPSG:4612'}},
    'coordinates': [138.62719, 36.32367]}},
  {'type': 'Feature',
   'id': 3,
   'properties': {'ii': '972004000070',
    'r': '国道１８号',
    'ad': '長野県北佐久郡軽井沢町軽井沢',
    'ot': '2024年10月29日 14:05',
  

In [13]:
# foliumマップを作成
center_lon, center_lat = 139.75492, 35.70252
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)
folium.TileLayer('cartodbpositron')
# フィーチャのジオメトリを描画
for feature in traffic_data['features']:
    geometry = feature['geometry']
    if geometry['type'] == 'MultiLineString':
        # 各ラインを描画
        for line_coords in geometry['coordinates']:
            # 座標の順序を逆にする（foliumは[lat, lon]の順で指定）
            lat_lon_coords = [[coord[1], coord[0]] for coord in line_coords]
            # PolyLineでラインを追加
            folium.PolyLine(
                locations=lat_lon_coords,
                color='red',
                weight=3,
                opacity=0.8
            ).add_to(m)

# マップを保存
m.save("C:/Users/tora2/downloads/traffic_map.html")

In [16]:
# foliumマップを作成
center_lon, center_lat = 139.75492, 35.70252
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)
# フィーチャのジオメトリを描画
for feature in traffic_data['features']:
    geometry = feature['geometry']
    if geometry['type'] == 'MultiLineString':
        for line_coords in geometry['coordinates']:
            # 座標の順序を逆にする（foliumは[lat, lon]の順で指定）
            lat_lon_coords = [[coord[1], coord[0]] for coord in line_coords]

            # PolyLineでラインを追加
            folium.PolyLine(
                locations=lat_lon_coords,
                color='red',
                weight=3,
                opacity=0.8,
                dash_array='5, 5',  # 破線を使う場合
                tooltip='Line Segment'
            ).add_to(m)

            # 矢印（中間点に追加）
            middle_point = lat_lon_coords[len(lat_lon_coords) // 2]
            folium.RegularPolygonMarker(
                location=middle_point,
                fill_color='red',
                number_of_sides=3,
                radius=8,
                rotation=0
            ).add_to(m)

            # 始点にマーカーを追加
            start_point = lat_lon_coords[0]
            folium.Marker(
                location=start_point,
                icon=folium.Icon(color='green', icon='play', prefix='fa'),
                tooltip='Start Point'
            ).add_to(m)

            # 終点にマーカーを追加
            end_point = lat_lon_coords[-1]
            folium.Marker(
                location=end_point,
                icon=folium.Icon(color='red', icon='stop', prefix='fa'),
                tooltip='End Point'
            ).add_to(m)

# マップを保存
m.save("C:/Users/tora2/downloads/enhanced_traffic_map.html")

In [22]:
# マップの中心座標（例として東京付近）
center_lon, center_lat = 139.37917, 35.71065

# foliumマップの作成
m = folium.Map(location=[center_lat, center_lon], zoom_start=20)

# ラインデータ（緯度経度のリスト）
line_coords = [[139.37917, 35.71065],
               [139.37898, 35.7102],
               [139.37893, 35.70995],
               [139.37893, 35.70995],
               [139.37876, 35.70907],
               [139.37863, 35.70833],
               [139.37863, 35.70833],
               [139.37842, 35.7072],
               [139.37842, 35.70716],
               [139.37834, 35.70671],
               [139.37875, 35.70729]]

# ラインを地図上に追加
folium.PolyLine(
    locations=[[line_coord[1], line_coord[0]] for line_coord in line_coords],  # 座標リスト（緯度, 経度の順）
    color='blue',           # ラインの色
    weight=5,               # ラインの太さ
    opacity=1             # ラインの透明度
).add_to(m)

# 地図を保存
m.save("C:/Users/tora2/downloads/simple_line_map.html")

In [53]:
# エンドポイントのURL
url = 'https://resas.go.jp/api/v1/cities/13'

# リクエストヘッダーの設定
headers = {
    'User-Agent': 'Mozilla/5.0',
    'Accept': 'application/json',
}

# GETリクエストの送信
response = requests.get(url, headers=headers)

# レスポンスの解析
if response.status_code == 200:
    data = response.json()
    # データを表示
    print(data)
else:
    print(f"エラー: {response.status_code}")

fo

エラー: 404
